# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

In [1]:
import nltk
nltk.download('wordnet')
nltk.download('stopwords')
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from scipy import stats
pd.options.mode.chained_assignment = None
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error, mean_absolute_error
import seaborn as sns
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import MinMaxScaler
import warnings
warnings.filterwarnings('ignore')
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import make_scorer
from sklearn.svm import SVR
from catboost import Pool, CatBoostRegressor, cv
from lightgbm import LGBMRegressor
import time
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from sklearn.metrics import f1_score
from sklearn.utils import class_weight
from sklearn.ensemble import RandomForestClassifier
import re
import scipy
import spacy
import lightgbm as lgb
from sklearn.linear_model import PassiveAggressiveClassifier

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\erofe\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\erofe\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Подготовка

In [2]:
#анализируем набор данных
filepath = r"C:\Users\erofe\Downloads\toxic_comments.csv"
df = pd.read_csv(filepath, sep=',')
#ищем дубликаты
print(df.duplicated().sum())
#выводим первые 10 строк таблицы
print(df.head(10))
#выводим информацию о таблице
print(df.info())

0
   Unnamed: 0                                               text  toxic
0           0  Explanation\nWhy the edits made under my usern...      0
1           1  D'aww! He matches this background colour I'm s...      0
2           2  Hey man, I'm really not trying to edit war. It...      0
3           3  "\nMore\nI can't make any real suggestions on ...      0
4           4  You, sir, are my hero. Any chance you remember...      0
5           5  "\n\nCongratulations from me as well, use the ...      0
6           6       COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK      1
7           7  Your vandalism to the Matt Shirvington article...      0
8           8  Sorry if the word 'nonsense' was offensive to ...      0
9           9  alignment on this subject and which are contra...      0
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 

с данными все хорошо, за исключением непонятного 1 столбца, который не был описан в условии.

In [3]:
#проверяем значения в столбце с оценкой
df['toxic'].unique()

array([0, 1], dtype=int64)

Значения корректны

In [4]:
#изучаем столбец с непонятными на 1 глаз данными
df['Unnamed: 0'].value_counts()

Unnamed: 0
0         1
106294    1
106287    1
106288    1
106289    1
         ..
53159     1
53160     1
53161     1
53162     1
159450    1
Name: count, Length: 159292, dtype: int64

Скорее всего этот столбец является просто повторением индексов строк, так что его можно удалить.

In [5]:
df = df.drop(['Unnamed: 0'], axis = 1)
df

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0
...,...,...
159287,""":::::And for the second time of asking, when ...",0
159288,You should be ashamed of yourself \n\nThat is ...,0
159289,"Spitzer \n\nUmm, theres no actual article for ...",0
159290,And it looks like it was actually you who put ...,0


In [6]:
#Очистим в тексте разделители строки и заглавные символы
def cleaning(text):
    text = re.sub(r"(?:\n|\r)", " ", text)
    text = re.sub(r"[^a-zA-Z ]+", "", text).strip()
    text = text.lower()
    return text

df['text'] = df['text'].apply(cleaning)

In [7]:
# Загрузка модели SpaCy для английского языка
nlp = spacy.load("en_core_web_sm")

# Функция для лемматизации текста с использованием SpaCy
def lemmatize_text_spacy(text):
    doc = nlp(text)
    return ' '.join([token.lemma_ for token in doc])

# Применение лемматизации к текстам
df['text_lemmatized'] = df['text'].apply(lemmatize_text_spacy)

Вывод: В данной части работы была проведена подготовка и обработка данных, данные были загружены и удален столбец, который не несёт смысла. Так же была проведена очистка и лемматизация текста.

## Обучение

In [8]:
# Создание TF-IDF векторизатора
vectorizer = TfidfVectorizer(stop_words=stopwords.words('english'))

# Вычисление TF-IDF значений для текстов
X = vectorizer.fit_transform(df['text_lemmatized'])
y = df['toxic']

# Разделение данных на обучающий и проверочный наборы
X_train, X_prom, y_train, y_prom = train_test_split(X, y, test_size=0.4, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_prom, y_prom, test_size=0.5, random_state=42)


# Определение параметров для GridSearchCV
param_grid = {
    'C': [0.1, 1, 4, 10, 100],
    'penalty': ['l1', 'l2']
}

# Инициализация модели
model = LogisticRegression(max_iter=100, random_state=12345, solver='liblinear')

# Поиск лучших параметров
grid_search = GridSearchCV(model, param_grid, scoring= 'f1', cv=5, n_jobs = -1)
grid_search.fit(X_train, y_train)

# Получение лучших параметров и оценка модели
best_params = grid_search.best_params_
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_val)
f1 = f1_score(y_val, y_pred)
print("Best Parameters:", best_params)
print("F1 Score:", f1)

Best Parameters: {'C': 4, 'penalty': 'l1'}
F1 Score: 0.7689655172413793


In [9]:
# Определение параметров для GridSearchCV
param_grid = {
    'C': [0.1, 1, 4, 10, 100],
    'loss': ['hinge', 'squared_hinge']
}

# Инициализация модели PassiveAggressiveClassifier
model1 = PassiveAggressiveClassifier(random_state=12345)

# Поиск лучших параметров
grid_search = GridSearchCV(model1, param_grid, scoring='f1', cv=5, n_jobs=-1)
grid_search.fit(X_train, y_train)

# Получение лучших параметров и оценка модели
best_params1 = grid_search.best_params_
best_model1 = grid_search.best_estimator_
y_pred = best_model1.predict(X_val)
f1 = f1_score(y_val, y_pred)
print("Best Parameters:", best_params1)
print("F1 Score:", f1)

Best Parameters: {'C': 0.1, 'loss': 'hinge'}
F1 Score: 0.7688117126319374


Лучшей моделью оказалась PassiveAggressiveClassifier, ее и будем тестировать.

In [10]:
# Создание модели с лучшими параметрами
best_model = PassiveAggressiveClassifier(random_state=12345, **best_params1)

# Объединение обучающих и валидационных данных
X_train_full = scipy.sparse.vstack([X_train, X_val])
y_train_full = pd.concat([y_train, y_val])

# Обучение модели на всех обучающих данных
best_model.fit(X_train_full, y_train_full)

# Предсказание на тестовых данных
y_test_pred = best_model.predict(X_test)

# Оценка качества модели на тестовых данных
f1_test = f1_score(y_test, y_test_pred)
print("F1 Score on Test Data:", f1_test)

F1 Score on Test Data: 0.7878057182225284


Модель показала достойное значение.

## Выводы

В 1 части работы была проведена загрузка и подготовкеа данных. Данные состояли из 3 столбцов(Один ненужный с индексами, который был удален, столбец с самим комментарием и столбец с определением токсичности(целевой признак)). Была так же проведена лемматизация и очистка текста. Во 2 части работы были обучены и протестированы 2 модели PassiveAggressiveClassifier и LogisticRegression, лучшей оказалась PassiveAggressiveClassifier, она и была использована на тестовых данных, где показала нужное нам значение метрики f1 (больше 0.75)